## Links

### https://stackoverflow.com/questions/10716464/what-are-the-correct-usage-parameter-values-for-houghcircles-in-opencv-for-iris

### https://stackoverflow.com/questions/9860667/writing-robust-color-and-size-invariant-circle-detection-with-opencv-based-on

## HOUGH Detect
### Parameters
- image	8-bit, single-channel, grayscale input image.
- circles	Output vector of found circles. Each vector is encoded as a 3-element floating-point vector (x,y,radius) .
- method	Detection method, see cv::HoughModes. Currently, the only implemented method is HOUGH_GRADIENT
- dp	Inverse ratio of the accumulator resolution to the image resolution. For example, if dp=1 , the accumulator has the same resolution as the input image. If dp=2 , the accumulator has half as big width and height.
- minDist	Minimum distance between the centers of the detected circles. If the parameter is too small, multiple neighbor circles may be falsely detected in addition to a true one. If it is too large, some circles may be missed.
- param1	First method-specific parameter. In case of CV_HOUGH_GRADIENT , it is the higher threshold of the two passed to the Canny edge detector (the lower one is twice smaller).
- param2	Second method-specific parameter. In case of CV_HOUGH_GRADIENT , it is the accumulator threshold for the circle centers at the detection stage. The smaller it is, the more false circles may be detected. Circles, corresponding to the larger accumulator values, will be returned first.
- minRadius	Minimum circle radius.
- maxRadius	Maximum circle radius.

## Get position vertex
### HoughCircles

In [72]:
import cv2 as cv
import math
import numpy as np


def drawcircles(circles, src):
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1])
            print('center=', center, end=' ')
            # circle center
            cv.circle(src, center, 1, (0, 100, 100), 3)
            # circle outline
            radius = i[2]
            print('radius=', radius, end='\n')
            cv.circle(src, center, radius, (255, 0, 255), 3)

def recognitioncircles(src, iter=3):
    # HOUGH DETECT
    # filters
    # GRAY
    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    # delete noise
    gaus = cv.GaussianBlur(gray, (5, 5), 2)
    # cv.imwrite("gaus.jpg", gaus)

    rows = gray.shape[0]
    h, w = src.shape[:2]

    # const
    par1 = 90 #90 #80 #40
    par2 = 70 #68 #50 #67
    minR = 2
    maxR = 63
    
    circles = cv.HoughCircles(gaus, cv.HOUGH_GRADIENT, 2, 2*maxR,
                          param1=par1, param2=par2,
                          minRadius=minR, maxRadius=maxR)
    
    drawcircles(circles, src)
    cv.imwrite("detect.jpg", src)
    
    
# Loads an image
src = cv.imread("2.jpg", cv.IMREAD_COLOR)
# Check if image is loaded fine
if src is None:
    print ('Error opening image!')
    print ('Usage: hough_circle.py [image_name -- default ' + file + '] \n')
    
recognitioncircles(src)

center= (219, 235) radius= 44
center= (91, 41) radius= 27


## MSER Blob Detector
### 1

In [183]:
import cv2
import math
import numpy as np

orig = cv2.imread("2.jpg")
img = orig.copy()
img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# kernel = np.ones((5,5), np.uint8)
# img2 = cv2.erode(img2, kernel, iterations = 1)
#img2 = cv2.GaussianBlur(img2,(5,5),2)
img2 = cv2.Canny(img2, 100, 255)

detector = cv2.MSER_create()
fs = detector.detect(img2)
fs.sort(key = lambda x: -x.size)

def supress(x):
        for f in fs:
                distx = f.pt[0] - x.pt[0]
                disty = f.pt[1] - x.pt[1]
                dist = math.sqrt(distx*distx + disty*disty)
                if (f.size > x.size) and (dist<f.size/2):
                        return True

sfs = [x for x in fs if not supress(x)]

for f in sfs:
        cv2.circle(img, (int(f.pt[0]), int(f.pt[1])), int(f.size/2), (150, 55, 65), 2, cv2.LINE_AA)
        cv2.circle(img, (int(f.pt[0]), int(f.pt[1])), int(f.size/2), (250, 200, 200), 1, cv2.LINE_AA)

h, w = orig.shape[:2]
vis = np.zeros((h, w*2+5), np.uint8)
vis = cv2.cvtColor(vis, cv2.COLOR_GRAY2BGR)
vis[:h, :w] = orig
vis[:h, w+5:w*2+5] = img

# cv2.imshow("image", vis)
cv2.imwrite("c_o.jpg", vis)
cv2.waitKey()
cv2.destroyAllWindows()

### 2 http://qaru.site/questions/1242600/why-houghcircles-returns-0-circles-while-trying-to-detect-irises

In [175]:
import cv2
import math
import numpy as np
import sys

def non_maximal_supression(x):
    for f in features:
        distx = f.pt[0] - x.pt[0]
        disty = f.pt[1] - x.pt[1]
        dist = math.sqrt(distx*distx + disty*disty)
        if (f.size > x.size) and (dist<f.size/2):
            return True

thresh = 70
img = cv2.imread("121.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#erosion = cv2.GaussianBlur(bw, (5, 5), 2)
bw = cv2.Canny(gray, 100, 255)
# erosion = cv2.medianBlur(bw, 5)
# kernel = np.ones((5,5), np.uint8)
# bw = cv2.erode(erosion, kernel, iterations = 2)

detector = cv2.MSER_create()
features = detector.detect(bw)
# coordinates, bboxes = detector.detectRegions(gray)

features.sort(key = lambda x: -x.size)

features = [ x for x in features if (x.size > 40)] 
reduced_features = [x for x in features if not non_maximal_supression(x)]

for rf in reduced_features:
    cv2.circle(img, (int(rf.pt[0]), int(rf.pt[1])), int(rf.size/2), (0,0,255), 3)

# cv2.imshow("iris detection", img)
cv2.imwrite("iris detection.jpg", img)
# cv2.waitKey()

-1

## Find the Center of a Blob (Centroid) using OpenCV
### https://www.learnopencv.com/find-center-of-blob-centroid-using-opencv-cpp-python/

In [ ]:
import cv2
import math
import numpy as np

# read image
img = cv2.imread("58.jpg")
 
# convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_image = cv2.Canny(gray, 100, 255)
# cv2.imshow('Canny',gray_image)
 
# convert the grayscale image to binary image
ret,thresh = cv2.threshold(gray_image,127,255,0)
# cv2.imshow("thresh", thresh)
 
# find contours in the binary image
im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cv2.imshow("im2", im2)

for c in contours:
   # calculate moments for each contour
    M = cv2.moments(c)
 
   # calculate x,y coordinate of center
    cX = 0
    cY = 0
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    
    cv2.circle(img, (cX, cY), 5, (255, 255, 255), -1)
#         cv2.putText(img, "c", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
 
# display the image
cv2.imshow("Image", img)
cv2.waitKey(0)

## Simple blob detector
### http://qaru.site/questions/180029/how-to-use-opencv-simpleblobdetector
### https://www.learnopencv.com/blob-detection-using-opencv-python-c/

In [ ]:
# Standard imports
import cv2
import numpy as np

# Read image
im = cv2.imread("63.jpg")

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 10
params.maxThreshold = 200

# Filter by Area.
params.filterByArea = True
params.minArea = 1500

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.01

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.81

# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01

# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)

# filters
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im = cv2.Canny(gray, 100, 255)

# Detect blobs.
keypoints = detector.detect(im)

# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
# the size of the circle corresponds to the size of blob

im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show blobs
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)

In [ ]:
# Standard imports
import cv2
import numpy as np

# Read image
im = cv2.imread("63.jpg")

detector = cv2.SimpleBlobDetector_create()

gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# kernel = np.ones((5,5), np.uint8)
# erosion = cv2.erode(gray, kernel, iterations = 1)
#thresh = cv2.GaussianBlur(gray, (5, 5), 2)
im = cv2.Canny(gray, 100, 255)

# Detect blobs.
keypoints = detector.detect(im)

# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
# the size of the circle corresponds to the size of blob

im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show blobs
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)